In [1]:
# Import Dependencies
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np
import requests
import time
from datetime import date
import calendar

In [2]:
# Import & Read CSV generated from DarkSky Weather API
csv_path1= "Resources/temp_weather.csv"
weather_df = pd.read_csv(csv_path1)
# Clean weather_df
del weather_df['Unnamed: 0']


In [3]:
# weather_df.count() shows Date has 906 non-empty rows 
#          while Temperature and Weather condition have 902 non empty rows.

# Should we add the lines: 
weather_df = weather_df.dropna(how = 'any') 
weather_df = weather_df.reset_index(drop = True)

In [4]:
# Import, Read and Clean Pittsburgh Crime Codes - non drug related
csv2_path= "Resources/crime_codes.csv"
crime_code_df = pd.read_csv(csv2_path)

crime_code_df.columns= ['crime code',
                         'description ', 'statutory class','§  303.3 OFFENSE GRAVITY SCORE ', '§  303.7']

cc_df = crime_code_df[['crime code', 'statutory class']].copy()

cc_df = cc_df.dropna(how = 'any')
cc_df = cc_df.reset_index(drop = True)

i = 0
while i < len(cc_df['crime code']):
    cc_df['crime code'][i] = cc_df['crime code'][i].replace('\xa0', '')
    cc_df['crime code'][i] = cc_df['crime code'][i].replace('Inchoate', '')
    
    cc_df['statutory class'][i] = cc_df['statutory class'][i].replace('\xa0', '')
    cc_df['statutory class'][i] = cc_df['statutory class'][i].replace('Inchoate', '')
    i+=1

for x in range(len(cc_df['statutory class'])):
    cc_df['statutory class'][x] = cc_df['statutory class'][x].split('-')[0]
    cc_df['statutory class'][x] = cc_df['statutory class'][x].split(' ')[0]
    if cc_df['statutory class'][x] == 'Murder':
        cc_df['statutory class'][x] = 'F'
        
classes = ['M', 'F']
classes = set(classes)

for x in range(len(cc_df['statutory class'])):
    if cc_df['statutory class'][x].split(' ')[0] not in classes:
        cc_df = cc_df.drop(x)

cc_df = cc_df.reset_index(drop = True)



In [ ]:
# Import, Read and Clean Pittsburgh Crime Codes - drug related

csv3_path= "Resources/drug_crime_code.csv"
crime_code_df2 = pd.read_csv(csv3_path)
crime_code_df2.columns = ['crime code', 'DESCRIPTION ', 'statutory class',
       '§  303.3 OFFENSE GRAVITY SCORE ', '§  303.7']
drug_cc_df = crime_code_df2[['crime code', 'statutory class']].copy()

drug_cc_df = drug_cc_df.dropna(how = 'any')
drug_cc_df = drug_cc_df.reset_index(drop = True)


i = 0
while i < len(drug_cc_df['crime code']):
    x = drug_cc_df['crime code'][i]
    drug_cc_df['crime code'][i] = f'13{x}'
    drug_cc_df['crime code'][i] = drug_cc_df['crime code'][i].replace('\xa0', '')
    drug_cc_df['crime code'][i] = drug_cc_df['crime code'][i].replace('Inchoate', '')
    
    drug_cc_df['statutory class'][i] = drug_cc_df['statutory class'][i].replace('\xa0', '')
    drug_cc_df['statutory class'][i] = drug_cc_df['statutory class'][i].replace('Inchoate', '')
    i+=1

    
for x in range(len(drug_cc_df['statutory class'])):
    drug_cc_df['statutory class'][x] = drug_cc_df['statutory class'][x].split('-')[0]
    drug_cc_df['statutory class'][x] = drug_cc_df['statutory class'][x].split(' ')[0]
    
for x in range(len(drug_cc_df['statutory class'])):
    if drug_cc_df['statutory class'][x].split(' ')[0] not in classes:
        drug_cc_df = drug_cc_df.drop(x)    

drug_cc_df = drug_cc_df.reset_index(drop = True)    

In [ ]:
#Import & Read Original Pittsburgh Crime Data CSV
csv_path3= "Resources/crime_data.csv"
master_df = pd.read_csv(csv_path3)


In [ ]:
# Clean master_df & Print Pittsburgh Crime DataFrame (organized_df)
# Split up ARRESTTIME column into Arrest Hour, Arrest Date, Arrest Month, Arrest Year, Arrest Unix Timestamp columns
master_df[['Date','Arrest_Time']]=master_df['ARRESTTIME'].str.split('T', expand=True,n=2)

# Drop unnecessary columns: PK, CCR, ARRESTLOCATION, OFFENSES, INCIDENTTRACT, COUNCIL_DISTRICT, PUBLIC_WORKS_DIVISION.
del master_df['PK']
del master_df['CCR']
del master_df['ARRESTTIME']
del master_df['ARRESTLOCATION']
del master_df['INCIDENTTRACT']
del master_df['COUNCIL_DISTRICT']
del master_df['PUBLIC_WORKS_DIVISION']
# Rename Columns
master_df.columns = ['Age', 'Gender', 'Race', 'Offenses Severity', 'Incident Location', 'Incident Neighborhood', 'Incident Zone', 'Incident Longitude', 'Incident Latitude', 'Date', 'Arrest_Time']
# Reorganize Columns
organized_df = master_df[['Age', 'Gender', 'Race', 'Offenses Severity', 'Date', 'Arrest_Time', 'Incident Location', 'Incident Latitude', 'Incident Longitude', 'Incident Neighborhood', 'Incident Zone']]
# View DataFrame
# organized_df.head(1)

In [ ]:
# Merge organized_df with weather_df
merged_df = weather_df.merge(organized_df, how="inner", on=["Date", "Date"])
# merged_df.head(1)

In [ ]:
# Clean & Organize merged_df
merged_df[['Arrest Year','Arrest Month','Arrest Date']]=merged_df['Date'].str.split('-', expand=True,n=2)
merged_df[['Arrest Hour','Arrest_LongTime']]=merged_df['Arrest_Time'].str.split(':', expand=True,n=1)

# Convert "Arrest Month" column from Month Numbers to Month Names
look_up = {'01': 'January', '02': 'February', '03': 'March', '04': 'April', '05': 'May',
            '06': 'June', '07': 'July', '08': 'August', '09': 'September', '10': 'October', '11': 'November', '12': 'December'}
merged_df['Arrest Month'] = merged_df['Arrest Month'].apply(lambda x: look_up[x])

# Convert "Arrest Timestamp" column into "Arrest Day" column
merged_df['Date'] = pd.to_datetime(merged_df.Date) 
merged_df['Arrest Day of Week'] = merged_df.Date.dt.weekday_name

# Create 'Arrest Time of Day' column from 'Arrest Hour' column
merged_df["Arrest Hour"] = pd.to_numeric(merged_df["Arrest Hour"])
# Create bins in which to place values based upon Time of Day
day_bins = [0, 4, 8, 12, 16, 20, 24]
# Create labels for day_bins
day_labels = ["Early Morning (12-3am)", "Morning (4-8am)", "Late Morning (9am-12pm)", "Early Afternoon (1-4pm)", "Early Evening (5-8pm)", "Evening (9-12am)"]
# Create 'Arrest Time of Day' column for storing bins
merged_df['Arrest Time of Day'] = pd.cut(merged_df['Arrest Hour'], day_bins, labels=day_labels)
merged_df["Temperature"] = pd.to_numeric(merged_df["Temperature"])
temp_bins = [0,10,20,30,40,50,60,70,80,90,100]
temp_labels = ['0s','10s','20s','30s','40s','50s','60s','70s','80s','90s']
merged_df['Temperature Range'] = pd.cut(merged_df['Temperature'],temp_bins, labels=temp_labels)
# Delete unwanted columns
del merged_df['Date']
del merged_df['Arrest_Time']
del merged_df['Arrest_LongTime']
# Reorganize columns
full_crime_df = merged_df[['Age', 'Gender', 'Race', 'Offenses Severity', 'Arrest Time of Day', 'Arrest Hour', 'Arrest Day of Week', 'Arrest Date', 'Arrest Month', 'Arrest Year','Temperature Range', 'Temperature', 'Weather Condition', 'Incident Location', 'Incident Latitude', 'Incident Longitude', 'Incident Neighborhood', 'Incident Zone']]

# Edit 'Age' column to have no person who is less than 1 year old and no one older than 100.
indexNames = full_crime_df[(full_crime_df['Age'] >= 100) & (full_crime_df['Age'] == 0)].index
full_crime_df.drop(indexNames , inplace=True)
# Get rid of Outliers
full_crime_df.dropna(inplace=True)


In [ ]:
#same recommendation for full_crime_df as made for weather_df

In [ ]:
full_crime_df = full_crime_df.dropna(how = 'any')

full_crime_df = full_crime_df[(full_crime_df.Age > 0) & 
                             (full_crime_df.Age < 100)]

full_crime_df = full_crime_df.reset_index(drop = True)


In [ ]:
##Start of breaking offenses up just using crime codes

off_df = full_crime_df[['Offenses Severity']].copy()
off_df = off_df.reset_index(drop = True)
off_df.columns = ['OFFENSES']

#looping through and counting the number of offenses per arrest, t1 is the count of offenses per arrest
t1 = []
i = 0
while i < len(off_df["OFFENSES"]):
    t1.append(len(off_df["OFFENSES"][i].split(' / ')))
    i += 1    
off_count = np.max(t1)  

#creating columns for max number of offenses for an individual arrest and the severity of that crime
i = 1
offs = []
while i <= off_count:
    add_col1 = f'Offense {i}'
    add_col2 = f'Severity {i}'
    off_df[add_col1] = ''
    off_df[add_col2] = ''
    i+=1
    
j = 0
while j < len(off_df['OFFENSES']):
    ind_off = off_df["OFFENSES"][j].split(' / ')
    k = 0
    while k < len(ind_off):
        off_col = f'Offense {k+1}'
        off_df[off_col][j] = ind_off[k].split(' ')[0]
        k += 1
    j += 1

# off_df.head()

In [ ]:

i = 0
while i < len(off_df['Offense 1']):
    j = 0
    
    while j < len(cc_df['crime code']):
        code_test1 = cc_df['crime code'][j]
        code_test2 = cc_df['crime code'][j].split('(')[0]
        if off_df['Offense 1'][i] == code_test1:
            off_df['Severity 1'][i] = cc_df['statutory class'][j]
            break
            
        elif off_df['Offense 1'][i] == code_test2:
            off_df['Severity 1'][i] = cc_df['statutory class'][j]
            break
            
  
        j += 1
    i+=1
    
x = 0
while x < len(off_df['Severity 1']):
    if off_df['Severity 1'][x] == "":
        j = 0
        
        while j < len(drug_cc_df['crime code']):
            code_test1 = drug_cc_df['crime code'][j]
            code_test2 = drug_cc_df['crime code'][j].split('(')[0]
            
            if off_df['Offense 1'][x] == code_test1:
                off_df['Severity 1'][x] = drug_cc_df['statutory class'][j]
                break
                
            elif off_df['Offense 1'][x] == code_test2:
                off_df['Severity 1'][x] = drug_cc_df['statutory class'][j]
                break
              
            j += 1
    x += 1


off_df.head()

In [ ]:
# misdem_ct = 0
# felon_ct = 0
# murd_ct = 0
# for x in off_df['Severity 1']:
#     t1 = x.split('-')[0]
#     t2 = x.split(' ')[0]
#     if t1 == 'M':
#         misdem_ct += 1
#     elif t1 == 'F':
#         felon_ct += 1
#     elif t2 == 'Murder':
#         murd_ct += 1
    
# missing = len(off_df['Offense 1'])- (misdem_ct + felon_ct + murd_ct)         
# print(f'We have {misdem_ct} crimes classified as a misdemeanor')
# print(f'We have {felon_ct} crimes classified as a felony')
# print(f'We have {murd_ct} crimes classified as a murder')
# print(f'We have {missing} crimes not categorized')

In [ ]:
crime_severity_df = full_crime_df.copy()
crime_severity_df['Offenses Severity'] = off_df['Severity 1']


for x in range(len(crime_severity_df['Offenses Severity'])):
    if crime_severity_df['Offenses Severity'][x].split(' ')[0] not in classes:
        crime_severity_df = crime_severity_df.drop(x)


crime_severity_df = crime_severity_df.dropna(how = 'any')


crime_severity_df = crime_severity_df.reset_index(drop = True)
crime_severity_df.head()



In [ ]:
crime_severity_df.to_csv('../clean_crime_data.csv')